In [1]:
import random
random.seed(42)
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

In [8]:
import numpy as np

class MyModel:
    def __init__(self, k1=1, k2=1, k3=1, a1=1, a2=1, a3=1, g1=1, g2=1, g3=1, n1=1, n2=1, n3=1, b1=1, b2=1, b3=1,dm1 = 1.143402097500176,dm2= 1.143402097500176,dm3= 1.143402097500176,dp1= 0.7833664565550977,dp2=0.7833664565550977,dp3=0.7833664565550977):
        self.k1 = k1
        self.k2 = k2
        self.k3 = k3
        self.a1 = a1
        self.a2 = a2
        self.a3 = a3
        self.g1 = g1
        self.g2 = g2
        self.g3 = g3
        self.n1 = n1
        self.n2 = n2
        self.n3 = n3
        self.b1 = b1
        self.b2 = b2
        self.b3 = b3
        self.dm1= dm1
        self.dm2 = dm2
        self.dm3=dm3
        self.dp1=dp1
        self.dp2=dp2
        self.dp3=dp3

    def model(self, variables, t):
        m1, p1, m2, p2, m3, p3 = variables
        dm1dt = -dm1 * m1 + (self.a1 / (1 + ((1 / self.k1) * p2) ** self.n1)) + self.g1
        dp1dt = (self.b1 * m1) - (self.dp1 * p1)
        dm2dt = -dm2 * m2 + (self.a2 / (1 + ((1 / self.k2) * p3) ** self.n2)) + self.g2
        dp2dt = (self.b2 * m2) - (self.dp2 * p2)
        dm3dt = -dm3 * m3 + (self.a3 / (1 + ((1 / self.k3) * p1) ** self.n3)) + self.g3
        dp3dt = (self.b3 * m3) - (self.dp3 * p3)
        return np.array([dm1dt, dp1dt, dm2dt, dp2dt, dm3dt, dp3dt]).flatten()

In [2]:

def model(variables, t, params):
    m1, p1, m2, p2, m3, p3 = variables
    k1,k2,k3,a1,a2,a3,g1,g2,g3,n1,n2,n3,b1,b2,b3,dm1,dm2,dm3,dp1,dp2,dp3=params
    #for now we are setting a, g, n, b dm, dp to be constant
    #dm1 = dm2 = dm3 =1.143402097500176
    #dp1 = dp2 = dp3 = 0.7833664565550977
    dm1dt = -dm1*m1 + (a1 / (1 + ((1/k1) * p2)**n1)) + g1
    dp1dt = (b1*m1) - (dp1*p1)
    dm2dt = -dm2*m2 + (a2 / (1 + ((1/k2) * p3)**n2)) + g2
    dp2dt = (b2*m2) - (dp2*p2)
    dm3dt = -dm3*m3 + (a3 / (1 + ((1/k3) * p1)**n3)) + g3
    dp3dt = (b3*m3)-(dp3*p3)
    return np.array([dm1dt, dp1dt, dm2dt, dp2dt, dm3dt, dp3dt]).flatten()

In [4]:
def generate_params_val(num_params,num_samples,fixed_values):
    fixed_arr=np.tile(fixed_values,(num_samples,1))
    if num_params==3:
        random_params=np.hstack((np.random.default_rng().uniform(0.01,250,size=(num_samples,3)),fixed_arr))
    elif num_params==15:
        random_params=np.hstack((np.random.default_rng().uniform(0.01,250,size=(num_samples,3)),np.random.default_rng().uniform(20.,40.,size=(num_samples,3)),np.random.default_rng().uniform(0.,5.,size=(num_samples,3)),np.random.default_rng().uniform(1.,5.,size=(num_samples,3)),np.random.default_rng().uniform(0.01,50.,size=(num_samples,3)),fixed_arr))
    elif num_params==6:
        random_params=np.hstack((np.random.default_rng().uniform(0.01,250,size=(num_samples,3)),np.random.default_rng().uniform(20.,40.,size=(num_samples,3)),fixed_arr))
    return(random_params)

In [5]:
def find_mean_std(random_params):
    """a function that returns the mean and standard deviation of the summary statistics of the simulated data, for the number of samples specified"""
#generate an empty np array to store simulated data
    num_samples=random_params.shape[0]
    simulated_arr=np.zeros((num_samples, 13))
    for i in range(num_samples):
        simulated_data=solve_ode(random_params[i,:])
        simulated_arr[i,:]=summarise(simulated_data)
    s_std=np.std(simulated_arr, axis=0)
    s_mean=np.mean(simulated_arr, axis=0)
    return(s_mean, s_std)

In [16]:
s=np.array([1. 1.])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (488527813.py, line 1)

In [6]:

def solve_ode(params):
    initial_conditions = np.array([0, 2, 0, 1, 0, 3])
    solution = odeint(model, initial_conditions, t=np.linspace(0,100,100), args=(params,)) # The initial value point should be the first element of this sequence
    return solution

timepoints = int(100)
third = int(timepoints / 3)
#truncated_traj = true_data[third:timepoints,:]

In [7]:
from scipy.signal import find_peaks
def summarise(solution):
    data=solution[:,0:6:2]
    """Calculate summary statistics
        requires solution to be the result of calling solve_ode[parameters]
        e.g. true_data
        Returns
        -------
        a 1D array of summary statistics containing the mean of solution, the log variance of solution, the autocorrelation at lag 10, cross correlation of mRNA and protein species separately, number of peaks
        """
    mean=np.mean(data, axis=0) #mean of mRNA species
    var=np.var(data, axis=0) #var
    auto_cor=np.corrcoef(data[10:,0],solution[:-10,0])[1,0] #autocorrelation at lag 10 for m1
    cor=np.corrcoef(data,data, rowvar=False)[:3,1:4] #cross correlation
    cor_coef=np.diag(cor)
    #returning cross correlation coefficients of mRNA to mRNA species, protein to protein species
    truncated_trajectory=data[33:,:] 
    peaks,_=zip(*(find_peaks(truncated_trajectory[:,i], height=0) for i in range(3))) 
    peak_count=np.array([len(p) for p in peaks]) #number of peaks 
    return(np.concatenate([mean, var, auto_cor.flatten(), cor_coef, peak_count.flatten()]))

In [17]:
def generate_normalising_statistics(num_params, num_samples, fixed_values):
    param_vals=generate_params_val(num_params,num_samples,fixed_values)
    return(find_mean_std(param_vals))

for params list, for each param, fixed value=.,. (6 params and 15 params respectively)

In [18]:
defined_param=[1.1977430229572492,1.5302375124759988,1.5608364378206137,0.7747339528650133,0.7511393265314563,0.7528339378453786]

In [19]:
m_15_1,s_15_1=generate_normalising_statistics(num_params=15,num_samples=10000,fixed_values=defined_param)

In [21]:
s_15_1

array([ 8.77137653,  7.15573613,  6.55463759, 15.18564563, 10.7006633 ,
        9.3441815 ,  0.61750055,  0.43738495,  0.45012339,  0.44630964,
        2.07412819,  2.07920136,  2.07907287])